<a href="https://colab.research.google.com/github/cheems/Any-file-to-Google-Drive/blob/master/Any_file_to_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<font color='blue'> Any File to Google Drive - Shared by [Cheems](https://github.com/cheems) </font>**


---
#Mount Google Drive with Rclone

In [0]:
#@markdown <h3>📝 Note: Run this before using Rclone.</h3>
#@markdown <h4>📝 Upload an rclone profile</4>
Setup_Time_Zone = False 

import os; from google.colab import files; from IPython.display import HTML, clear_output

def upload_conf():
  try:
    display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Please upload the config file of rclone (rclone.conf) from your computer.</h2><br>"))
    UploadConfig = files.upload().keys()
    clear_output(wait=True)
    if len(UploadConfig) == 0:
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
    elif len(UploadConfig) == 1:
      for fn in UploadConfig:
        if os.path.isfile("/content/" + fn) == True:
          os.environ["rclone_conf"] = fn
          !mv -f "$rclone_conf" /root/.rclone.conf
          !chmod 666 /root/.rclone.conf
          if Setup_Time_Zone == True:
            !sudo dpkg-reconfigure tzdata
            clear_output(wait=True)
          if os.path.isfile("/usr/bin/rclone") == True:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">Config has been changed.</h2><br></center>"))
          else:
            !rm -rf /content/sample_data/
            !curl -s https://rclone.org/install.sh | sudo bash
            clear_output(wait=True)
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Installation has been successfully completed.</h2><br></center>"))
        else:
          return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
    else:
      for fn in UploadConfig:
        os.environ["rclone_conf"] = fn
        !rm -f "$rclone_conf"
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
  except:
    clear_output(wait=True)
    return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))

upload_conf()

---
# <img src='https://szyha.github.io/RcloneLabArchive/img/title_jdownloader.png' height="45" alt="JDownloader"/>

In [0]:
#@markdown <h3>⬅️ Click Here to Install JDownloader</h3>

import os, uuid, re, IPython
import ipywidgets as widgets

from glob import glob
from IPython.display import HTML, clear_output
from google.colab import output

Email = widgets.Text(placeholder="*Required", description="Email:")
Password = widgets.Text(placeholder="*Required", description="Password:")
Device = widgets.Text(placeholder="Optional", description="Name:")
SavePath = widgets.Text(placeholder="/content/Downloads", description="Save Path:")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def RefreshPath():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePath.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePath.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePath.options = ["/content", "/content/Downloads"]

def LoginForm():
  clear_output(wait=True)
  Email.value = ""
  Password.value = ""
  Device.value = ""
  RefreshPath()
  display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">If you don't have an account yet, please register <a href=\"https://my.jdownloader.org/login.html#register\" target=\"_blank\">here</a>.</h3>"), HTML("<br>"), Email, Password, Device, SavePath, MakeButton("Refresh", RefreshPath, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML(""))
  display(HTML("<br>"), MakeButton("Login", CheckLogin, "info"))
  if os.path.isfile("/root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json"):
    display(MakeButton("Cancel", Show, "danger"))
  
def RestartForm():
  clear_output(wait=True)
  display(MakeLabel("Restart Confirm?", ""), MakeButton("Confirm", Restart, "danger"), MakeButton("Cancel", Show, "warning"))
  
def ExitForm():
  clear_output(wait=True)
  display(MakeLabel("Exit Confirm?", ""), MakeButton("Confirm", Exit, "danger"), MakeButton("Cancel", Show, "warning"))

def CheckLogin():
  try:
    if not Email.value.strip():
      ERROR = "Email field is empty."
      THROW_ERROR
    if not "@" in Email.value and not "." in Email.value:
      ERROR = "Email is an incorrect format."
      THROW_ERROR
    if not Password.value.strip():
      ERROR = "Password field is empty."
      THROW_ERROR
    if not bool(re.match("^[a-zA-Z0-9]+$", Device.value)) and Device.value.strip():
      ERROR = "Only alphanumeric are allowed for the device name."
      THROW_ERROR
    Login()
  except:
    print(ERROR)

def Login():
  clear_output(wait=True)
  if SavePath.value == "/content":
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"/content\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  elif SavePath.value == "/content/Downloads":
    get_ipython().system_raw("mkdir -p -m 666 /content/Downloads")
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"/content/Downloads\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  else:
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"" + SavePath.value + "\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  if Device.value.strip() == "":
    Device.value = Email.value
  get_ipython().system_raw("pkill -9 -e -f java")
  get_ipython().system_raw("echo '{\"email\" : \"'" + Email.value + "'\", \"password\" : \"'" + Password.value + "'\", \"devicename\" : \"'" + Device.value + "'\", \"directconnectmode\" : \"LAN\"}' > /root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json")
  get_ipython().system_raw("java -jar /root/.JDownloader/JDownloader.jar -norestart -noerr -r &")
  Show()

def Restart():
  get_ipython().system_raw("pkill -9 -e -f java")
  get_ipython().system_raw("java -jar /root/.JDownloader/JDownloader.jar -norestart -noerr -r &")
  Show()
  
def Exit():
  get_ipython().system_raw("pkill -9 -e -f java")
  clear_output(wait=True)
  display(MakeButton("Start", Restart, "info"))
  
def Show():
  clear_output(wait=True)
  display(MakeLabel("Control Panel", ""), HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">You can login to the WebUI by clicking <a href=\"https://my.jdownloader.org/\" target=\"_blank\">here</a>.</h3>"), HTML("<h4 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">If the server didn't showup in 30 sec. please re-login.</h4>"), HTML("<br>"), MakeButton("Re-Login", LoginForm, "info"), MakeButton("Restart", RestartForm, "warning"), MakeButton("Exit", ExitForm, "danger"))
  
    
if not os.path.isfile("/root/.JDownloader/JDownloader.jar"):
  clear_output(wait=True)
  display(MakeLabel("Installing in Progress", "warning"))
  get_ipython().system_raw("rm -rf /content/sample_data/ && apt install openjdk-11-jre-headless -qq -y && mkdir -p -m 666 /root/.JDownloader && wget -q http://installer.jdownloader.org/JDownloader.jar -O /root/.JDownloader/JDownloader.jar && java -jar /root/.JDownloader/JDownloader.jar -norestart -h")
  LoginForm()
elif not os.path.isfile("/root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json"):
  LoginForm()
else:  
  Show()

---
# <img src='https://szyha.github.io/RcloneLabArchive/img/title_rclonelab.png' height="45" alt="RcloneLab"/>

In [0]:
# ============================= FORM ============================= #
Mode = "Move" #@param ["Off","Config", "Copy", "Move", "Sync", "Checker", "Deduplicate", "Remove Empty Directories", "Empty Trash", "qBittorrent"]
Compare = "Size & Mod-Time" #@param ["Size & Mod-Time", "Size & Checksum", "Only Mod-Time", "Only Size", "Only Checksum"]
Source = "/content/Downloads" #@param {type:"string"}
Destination = "YourDrive:DownloadPath" #@param {type:"string"}
Transfers = 4 #@param {type:"slider", min:1, max:20, step:1}
Checkers = 20 #@param {type:"slider", min:1, max:40, step:1}
#@markdown ---

#@markdown <center><h3><font color="#3399ff"><b>⚙️ Global Configuration ⚙️</b></font></h3></center>

#@markdown ---
Simple_Ouput = True #@param {type:"boolean"}
Skip_files_that_are_newer_on_the_destination = True #@param {type:"boolean"}
Skip_all_files_that_exist = True #@param {type:"boolean"}
Do_not_cross_filesystem_boundaries = False
Do_not_update_modtime_if_files_are_identical = False #@param {type:"boolean"}
Large_amount_of_files_optimization = False
Google_Drive_optimization = True #@param {type:"boolean"}
Dry_Run = False #@param {type:"boolean"}
Output_Log_File = "DEBUG" #@param ["OFF", "NOTICE", "INFO", "ERROR", "DEBUG"]
Extra_Arguments = "" #@param {type:"string"}
#@markdown ---

#@markdown <center><h3><font color="#3399ff"><b>↪️ Sync Configuration ↩️</b></font></h3></center>

#@markdown ---
Sync_Mode = "Delete during transfer" #@param ["Delete during transfer", "Delete before transfering", "Delete after transfering"]
Track_Renames = False #@param {type:"boolean"}
#@markdown ---

#@markdown <center><h3><font color="#3399ff"><b>💞 Deduplicate Configuration 💞</b></font></h3></center>

#@markdown ---
Deduplicate_Mode = "Interactive" #@param ["Interactive", "Skip", "First", "Newest", "Oldest", "Largest", "Rename"]
Deduplicate_Use_Trash = True #@param {type:"boolean"}
# ================================================================ #

import os; from IPython.display import HTML, clear_output
import sys


if Mode != "Off":

  # Optimized for Google Colaboratory
  os.environ["bufferC"] = "--buffer-size 96M"

  if Compare == "Size & Checksum":
    os.environ["compareC"] = "-c"
  elif Compare == "Only Mod-Time":
    os.environ["compareC"] = "--ignore-size"
  elif Compare == "Only Size":
    os.environ["compareC"] = "--size-only"
  elif Compare == "Only Checksum":
    os.environ["compareC"] = "-c --ignore-size"
  else:
    os.environ["compareC"] = ""

  os.environ["sourceC"] = Source
  os.environ["destinationC"] = Destination
  os.environ["transfersC"] = "--transfers "+str(Transfers)
  os.environ["checkersC"] = "--checkers "+str(Checkers)

  if Skip_files_that_are_newer_on_the_destination == True:
    os.environ["skipnewC"] = "-u"
  else:
    os.environ["skipnewC"] = ""

  if Skip_all_files_that_exist == True:
    os.environ["skipexistC"] = "--ignore-existing"
  else:
    os.environ["skipexistC"] = ""

  if Do_not_cross_filesystem_boundaries == True:
    os.environ["nocrossfilesystemC"] = "--one-file-system"
  else:
    os.environ["nocrossfilesystemC"] = ""

  if Do_not_update_modtime_if_files_are_identical == True:
    os.environ["noupdatemodtimeC"] = "--no-update-modtime"
  else:
    os.environ["noupdatemodtimeC"] = ""

  if Large_amount_of_files_optimization == True:
    os.environ["filesoptimizeC"] = "--fast-list"
  else:
    os.environ["filesoptimizeC"] = ""

  if Google_Drive_optimization == True:
    os.environ["driveoptimizeC"] = "--drive-chunk-size 32M --drive-acknowledge-abuse --drive-keep-revision-forever --disable copy"
  else:
    os.environ["driveoptimizeC"] = ""

  if Dry_Run == True:
    os.environ["dryrunC"] = "-n"
  else:
    os.environ["dryrunC"] = ""

  if Output_Log_File != "OFF":
    os.environ["statsC"] = "--log-file=/root/.rclone_log/rclone_log.txt"
  else:
    if Simple_Ouput == True:
      os.environ["statsC"] = "-v --stats-one-line --stats=5s"
    else:
      os.environ["statsC"] = "-v --stats=5s"

  if Output_Log_File == "INFO":
    os.environ["loglevelC"] = "--log-level INFO"
  elif Output_Log_File == "ERROR":
    os.environ["loglevelC"] = "--log-level ERROR"
  elif Output_Log_File == "DEBUG":
    os.environ["loglevelC"] = "--log-level DEBUG"
  else:
    os.environ["loglevelC"] = ""

  os.environ["extraC"] = Extra_Arguments

  if Sync_Mode == "Delete during transfer":
    os.environ["syncmodeC"] = "--delete-during"
  elif Sync_Mode == "Delete before transfering":
    os.environ["syncmodeC"] = "--delete-before"
  elif Sync_Mode == "Delete after transfering":
    os.environ["syncmodeC"] = "--delete-after"

  if Track_Renames == True:
    os.environ["trackrenamesC"] = "--track-renames"
  else:
    os.environ["trackrenamesC"] = ""

  if Deduplicate_Mode == "Interactive":
    os.environ["deduplicateC"] = "interactive"
  elif Deduplicate_Mode == "Skip":
    os.environ["deduplicateC"] = "skip"
  elif Deduplicate_Mode == "First":
    os.environ["deduplicateC"] = "first"
  elif Deduplicate_Mode == "Newest":
    os.environ["deduplicateC"] = "newest"
  elif Deduplicate_Mode == "Oldest":
    os.environ["deduplicateC"] = "oldest"
  elif Deduplicate_Mode == "Largest":
    os.environ["deduplicateC"] = "largest"
  elif Deduplicate_Mode == "Rename":
    os.environ["deduplicateC"] = "rename"

  if Deduplicate_Use_Trash == True:
    os.environ["deduplicatetrashC"] = ""
  else:
    os.environ["deduplicatetrashC"] = "--drive-use-trash=false"

  ### rclone Execution

  if Output_Log_File != "OFF" and Mode != "Config":
    !mkdir -p -m 666 /root/.rclone_log/
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Logging enables, rclone will not output log through the terminal, please wait until finished.</h2><br></center>"))


  if Mode == "Config":
    !rclone --config=/root/.rclone.conf config
  elif Mode == "Copy":
    !rclone --config=/root/.rclone.conf copy "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
  elif Mode == "Move":
    !rclone --config=/root/.rclone.conf move "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
  elif Mode == "Sync":
    !rclone --config=/root/.rclone.conf sync "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $syncmodeC $trackrenamesC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
  elif Mode == "Checker":
    !rclone --config=/root/.rclone.conf check "$sourceC" "$destinationC" $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
  elif Mode == "Deduplicate":
    !rclone --config=/root/.rclone.conf dedupe "$sourceC" $checkersC $statsC $loglevelC --dedupe-mode $deduplicateC $deduplicatetrashC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
  elif Mode == "Remove Empty Directories":
    !rclone --config=/root/.rclone.conf rmdirs "$sourceC" $statsC $loglevelC $dryrunC $extraC
  elif Mode == "Empty Trash":
    !rclone --config=/root/.rclone.conf cleanup "$sourceC" $statsC $loglevelC $dryrunC $extraC
  elif Mode == "qBittorrent":
    !chmod -R 666 /content/qBittorrent/
    !rclone --config=/root/.rclone.conf move "/content/qBittorrent/" "$destinationC" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs --exclude **/.unwanted/ $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC

  ### Log Output

  if Output_Log_File != "OFF" and Mode != "Config":
    ### Rename log file and output settings.
    !mv /root/.rclone_log/rclone_log.txt /root/.rclone_log/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).txt
    with open("/root/.rclone_log/" + Mode + "_settings.txt", "w") as f:
      f.write("Mode: " + Mode + \
              "\nCompare: " + Compare + \
              "\nSource: \"" + Source + \
              "\"\nDestination: \"" + Destination + \
              "\"\nTransfers: " + str(Transfers) + \
              "\nCheckers: " + str(Checkers) + \
              "\nSkip files that are newer on the destination: " + str(Skip_files_that_are_newer_on_the_destination) + \
              "\nSkip all files that exist: " + str(Skip_all_files_that_exist) + \
              "\nDo not cross filesystem boundaries: " + str(Do_not_cross_filesystem_boundaries) + \
              "\nDo not update modtime if files are identical: " + str(Do_not_update_modtime_if_files_are_identical) + \
              "\nDry-Run: " + str(Dry_Run) + \
              "\nOutput Log Level: " + Output_Log_File + \
              "\nExtra Arguments: \"" + Extra_Arguments + \
              "\"\nSync Moden: " + Sync_Mode + \
              "\nTrack Renames: " + str(Track_Renames) + \
              "\nDeduplicate Mode: " + Deduplicate_Mode + \
              "\nDeduplicate Use Trash: " + str(Deduplicate_Use_Trash))
    ### Compressing log file.
    !rm -f /root/rclone_log.zip
    !zip -r -q -j -9 /root/rclone_log.zip /root/.rclone_log/
    !rm -rf /root/.rclone_log/
    !mkdir -p -m 666 /root/.rclone_log/
    ### Send Log
    if os.path.isfile("/root/rclone_log.zip") == True:
      try:
        files.download("/root/rclone_log.zip")
        !rm -f /root/rclone_log.zip
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Sending log to your browser...</h2><br></center>"))
      except:
        !mv /root/rclone_log.zip /content/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).zip
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">You can use file explorer to download the log file.</h2><br></center>"))
    else:
      clear_output(wait=True)
      display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">There is no log file.</h2><br></center>"))

  ### Operation has been successfully completed.
  if os.path.isfile("/usr/bin/rclone") == False:
    clear_output(wait=True)
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ Please run the installation cell above first.</h2><br></center>"))
  elif Mode != "Config":
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">✅ Operation has been successfully completed.</h2><br></center>"))
else:
  !echo "doing nothing because the mode is set to off"

# 📚 Follow me on GitHub for more: [_@cheems_](https://github.com/cheems)
# 📦 Repo: [Github](https://github.com/cheems/Any-file-to-Google-Drive)